In [ ]:
DROP TABLE IF EXISTS PROJECT2.raw_data.cultural_facilities;

CREATE TABLE PROJECT2.raw_data.cultural_facilities (
    id INT,
    category STRING,
    facility_name STRING,
    address STRING,
    latitude FLOAT,
    longitude FLOAT,
    phone_number STRING,
    fax_number STRING,
    website STRING,
    opening_hours STRING,
    admission_fee STRING,
    closing_days STRING,
    opening_date DATE,
    seating_capacity INT,
    representative_image STRING,
    additional_info STRING,
    facility_description STRING,
    free_admission STRING,
    subway_stations STRING,
    bus_stations STRING,
    yellow_bus STRING,
    green_bus STRING,
    blue_bus STRING,
    red_bus STRING,
    airport_bus STRING
);

In [ ]:
COPY INTO PROJECT2.raw_data.cultural_facilities
FROM 's3://devcourse-project2/culture_facilities/서울시 문화공간 정보.csv'
credentials = (AWS_KEY_ID='' AWS_SECRET_KEY='')
FILE_FORMAT = (TYPE = 'CSV', FIELD_OPTIONALLY_ENCLOSED_BY = '"', SKIP_HEADER = 1)
ON_ERROR = 'CONTINUE';

In [ ]:
-- 기존 테이블이 있으면 삭제
DROP TABLE IF EXISTS PROJECT2.ANALYTICS.cultural_facilities_analytics;

-- 새 테이블 생성
CREATE TABLE PROJECT2.ANALYTICS.cultural_facilities_analytics AS
SELECT                    
    category,              
    facility_name,         
    address,               
    SPLIT_PART(address, ' ', 2) AS district,
    LONGITUDE,
    LATITUDE
FROM PROJECT2.raw_data.cultural_facilities;

In [ ]:
DROP TABLE IF EXISTS PROJECT2.raw_data.theaters;

CREATE TABLE PROJECT2.raw_data.theaters (
    administrative_code VARCHAR(20),
    management_number VARCHAR(50),
    permit_date VARCHAR(50),
    permit_cancel_date VARCHAR(50),
    business_status_code VARCHAR(10),
    business_status_name VARCHAR(50),
    detailed_status_code VARCHAR(10),
    detailed_status_name VARCHAR(50),
    closure_date VARCHAR(50),
    suspension_start_date VARCHAR(50),
    suspension_end_date VARCHAR(50),
    reopening_date VARCHAR(50),
    phone_number VARCHAR(20),
    area VARCHAR(20),
    postal_code VARCHAR(10),
    land_address VARCHAR(255),
    road_address VARCHAR(255),
    road_postal_code VARCHAR(10),
    business_name VARCHAR(100),
    last_updated_date VARCHAR(50),
    data_update_type VARCHAR(50),
    data_update_date VARCHAR(50),
    business_type VARCHAR(100),
    coord_x VARCHAR(20),
    coord_y VARCHAR(20),
    cultural_sports_category VARCHAR(50),
    cultural_business_type VARCHAR(50),
    total_floors VARCHAR(10),
    surrounding_environment VARCHAR(50),
    production_items TEXT,
    facility_area VARCHAR(20),
    above_ground_floors VARCHAR(10),
    underground_floors VARCHAR(10),
    building_usage VARCHAR(50),
    passage_width VARCHAR(10),
    lighting_intensity VARCHAR(10),
    karaoke_rooms VARCHAR(10),
    youth_rooms VARCHAR(10),
    emergency_stairs VARCHAR(10),
    emergency_exit VARCHAR(10),
    automatic_ventilation VARCHAR(10),
    youth_room_availability VARCHAR(10),
    special_lighting VARCHAR(10),
    soundproofing VARCHAR(10),
    video_replay_device VARCHAR(10),
    lighting_availability VARCHAR(10),
    sound_equipment VARCHAR(10),
    convenience_facility VARCHAR(10),
    fire_safety_facility VARCHAR(10),
    total_game_machines VARCHAR(10),
    other_business_types VARCHAR(100),
    provided_game_name TEXT,
    theater_type VARCHAR(50),
    item_name VARCHAR(100),
    initial_registration_date VARCHAR(50),
    regional_category VARCHAR(50)
);

In [ ]:
COPY INTO PROJECT2.raw_data.theaters
FROM 's3://devcourse-project2/culture_facilities/서울시 영화상영관 인허가 정보.csv'
CREDENTIALS = (AWS_KEY_ID='' AWS_SECRET_KEY='')
FILE_FORMAT = (TYPE = 'CSV', FIELD_OPTIONALLY_ENCLOSED_BY = '"', SKIP_HEADER = 1, EMPTY_FIELD_AS_NULL = TRUE)
ON_ERROR = 'CONTINUE';


In [ ]:
-- 기존 테이블이 있으면 삭제
DROP TABLE IF EXISTS PROJECT2.ANALYTICS.theaters_analytics;

-- 새 테이블 생성
CREATE TABLE PROJECT2.ANALYTICS.theaters_analytics AS
SELECT
    '영화관' AS category,
    business_name AS FACILITY_AREA,                
    road_address AS address, -- Using road_address for the address
    SPLIT_PART(road_address, ' ', 2) AS district,  
FROM PROJECT2.raw_data.theaters
WHERE business_status_name != '폐업'
GROUP BY administrative_code,business_name,ADDRESS;

In [ ]:
CREATE TABLE PROJECT2.ANALYTICS.CULTURAL_FACILITIES_INCLUDE_THEATER AS
SELECT *                     
FROM PROJECT2.ANALYTICS.CULTURAL_FACILITIES_ANALYTICS
UNION ALL
SELECT * 
FROM PROJECT2.ANALYTICS.THEATERS_ANALYTICS;

In [ ]:
CREATE TABLE PROJECT2.ANALYTICS.SEOUL_DISTRICT_PRICE_AND_FACILITY_SUMMARY AS
SELECT 
    A.DISTRICT AS "지역", 
    B.AVERAGE_PRICE_PER_UNIT AS "평단가",
    A.CATEGORY_COUNT AS "전체갯수",
    A.theater AS "영화관",
    A.library AS "도서관",
    A.concert AS "공연장",
    A.museum AS "박물관_기념관",
    A.art AS "미술관_갤러리",
    A.art1 AS "문화예술회관",
    A.art2 AS "문화원",
    A.etc AS "기타"
FROM 
    (SELECT DISTRICT, COUNT(*) AS CATEGORY_COUNT,
        SUM(CASE WHEN CATEGORY = '영화관' THEN 1 ELSE 0 END) AS theater,
        SUM(CASE WHEN CATEGORY = '도서관' THEN 1 ELSE 0 END) AS library,
        SUM(CASE WHEN CATEGORY = '공연장' THEN 1 ELSE 0 END) AS concert,
        SUM(CASE WHEN CATEGORY = '박물관/기념관' THEN 1 ELSE 0 END) AS museum,
        SUM(CASE WHEN CATEGORY = '미술관/갤러리' THEN 1 ELSE 0 END) AS art,
        SUM(CASE WHEN CATEGORY = '문화예술회관' THEN 1 ELSE 0 END) AS art1,
        SUM(CASE WHEN CATEGORY = '문화원' THEN 1 ELSE 0 END) AS art2,
        SUM(CASE WHEN CATEGORY = '기타' THEN 1 ELSE 0 END) AS etc
     FROM PROJECT2.ANALYTICS.CULTURAL_FACILITIES_INCLUDE_THEATER 
     GROUP BY DISTRICT) AS A
JOIN 
    (SELECT "자치구명" AS DISTRICT, "평단가" AS AVERAGE_PRICE_PER_UNIT 
     FROM PROJECT2.ANALYTICS.PROPERTY_UNIT_PRICE 
     WHERE "접수연도" = '2024') AS B
ON A.DISTRICT = B.DISTRICT;